In [ ]:
%aiida
import numpy as np
import scipy.constants as const
import ase.io
import subprocess
from copy import copy

from aiida.orm.data.parameter import ParameterData
from aiida.orm.data.structure import StructureData
from aiida.orm.data.folder import FolderData
from aiida.orm.calculation.work import WorkCalculation
from aiida_cp2k.calculations import Cp2kCalculation

import pickle
import ipywidgets as ipw
import re
import time

bohr = 1./0.52917720859

pp = [4.13643, 1.33747, 115.82004, 2.206835, 113.96850411, 5.84114,
      4.13643, 1.33747, 115.82004, 2.206835, 113.96850411, 5.84114,
      4.13643, 1.33747, 115.82004, 2.206835, 113.96850411, 5.84114]

## Parse AiiDA-cp2k calculations

In [ ]:
def parse(output):
    # Parses the forces.xyz file
    start = output.find('# Atom   Kind   Element          X              Y              Z')+len('# Atom   Kind   Element          X              Y              Z')+3
    end = output.find('\n SUM OF ATOMIC FORCES')
    out_forces = output[start:end].split('\n')
    return [map(np.float128, x.split()[3:]) for x in out_forces]

In [ ]:
def get_structures(mol='ft1-hexabenzocoronene'):
    # Get all the StructureData and Cp2kCalculation from the database
    qb = QueryBuilder()
    qb.append(StructureData,
              filters = {
                  'label': {'==': 'molecular circus'}
              },
              tag = 'mc',
              project = '*'
    )
    qb.append(WorkCalculation,
              descendant_of = 'mc',
              tag = 'mc_work',
              project = '*',
    )
    qb.append(Cp2kCalculation,
              output_of = 'mc_work',
              project = '*',
              filters = {
                  'and': [
                      { 'description': {'like': '{}%'.format(mol)} },
                      { 'label': {'like': 'ft_ene%'} },
                      { 'attributes.state': {'==': 'FINISHED'} }
                  ]
              }
    )

    r = qb.all()
    return [x[0] for x in r], [x[2] for x in r] # StructureData, Cp2kCalculation

## Find all the ft_ene.. Cp2kCalculations and extract energies & forces

In [ ]:
mol = 'ft1-hexabenzocoronene'
mol2 = 'ft2-precursor'
structures, cp2kcalc = get_structures(mol)
structures2, cp2kcalc2 = get_structures(mol2)


def get_atoms(structures):
    atoms = []
    for x, s in enumerate(structures):
        atoms.append(s.get_ase())
        print x
    return atoms

def get_dft_forces(cp2kcalc):
    forces_full = []
    energies_full = []
    forces_gas = []
    energies_gas = []
    for c in cp2kcalc:
        if c.label == 'ft_ene':
            f = c.get_outputs(FolderData)[0]
            forces_full.append(np.array(parse(f.get_file_content('aiida-forces-1_0.xyz')), dtype=np.float128)[:-1568])
            energies_full.append(c.get_outputs(ParameterData)[0].get_attr('energy'))
        if c.label == 'ft_ene_gas':
            f = c.get_outputs(FolderData)[0]
            forces_gas.append(np.array(parse(f.get_file_content('aiida-forces-1_0.xyz')), dtype=np.float128))
            energies_gas.append(c.get_outputs(ParameterData)[0].get_attr('energy'))
        
    return [np.array(forces_full),
            np.array(forces_gas),
            np.array(energies_full),
            np.array(energies_gas)]

#atoms = get_atoms(structures)
dft_forces_full1, dft_forces_gas1, dft_energies_full1, dft_energies_gas1 = get_dft_forces(cp2kcalc)
dft_forces_full2, dft_forces_gas2, dft_energies_full2, dft_energies_gas2 = get_dft_forces(cp2kcalc2)

dft_energy_slab = np.float128(-44834.879829141653317)

## Pickle them

In [ ]:
with open('pickling-ft1-hexabenzocoronene', 'r') as f:
    atoms1, [dft_forces_full1, dft_forces_gas1,
             dft_energies_full1, dft_energies_gas1] = pickle.load(f)

with open('pickling-ft2-precursor', 'r') as f:
    atoms2, [dft_forces_full2, dft_forces_gas2,
             dft_energies_full2, dft_energies_gas2] = pickle.load(f)


In [ ]:
def pickleDump():
    pickle_fn = 'pickling-'+mol
    with open(pickle_fn, 'wb') as f:
        pickle.dump([atoms[0:60], [dft_forces_full1, dft_forces_gas1, dft_energies_full1, dft_energies_gas1]], f)

    pickle_fn = 'pickling-'+mol2
    with open(pickle_fn, 'wb') as f:
        pickle.dump([atoms[60:], [dft_forces_full2, dft_forces_gas2, dft_energies_full2, dft_energies_gas2]], f)

In [ ]:
forces_fac1 = dft_forces_full1-dft_forces_gas1
forces_fac2 = dft_forces_full2-dft_forces_gas2

energy_fac1 = dft_energies_full1-dft_energies_gas1-dft_energy_slab
energy_fac2 = dft_energies_full2-dft_energies_gas2-dft_energy_slab

## Calculate potential

In [ ]:
def potential(xyz, A, av, B, ac, C, R):
    # calculate the potential for the distances xyz
    # and the parameters A, av, B, ac, C, R
    # result: pot = len(xyz)
    #         force = len(xyz)*3
    total_pot = 0
    total_force = np.zeros(3)
    for i, slab_atom in enumerate(xyz):
        r = np.linalg.norm(slab_atom) # 1x1
        if r>15*bohr:
            continue

        # Calculate the potential
        rtothe6 = np.power(r, -6)
        etotherR = np.exp(20.*r/R)
        etothe20 = np.exp(20)
        etotherR20 = etothe20/etotherR # e^(20-20r/R)
        
        pot = A*np.exp(-av*r)+B*np.exp(-ac*r)-C*rtothe6/(1+etotherR20)
        total_pot += pot
        
        # Calculate the force analytically
        first = A*np.exp(-av*r)
        second = B*np.exp(-ac*r)
        third = C*rtothe6/(1+etotherR20)
        
        rtothe7 = np.power(r, -7)
        xyzoverr = slab_atom/r
        force_third = third*20./R*etotherR20/(1+etotherR20)
        force_fourth = third*6./r

        force = (-av*first-ac*second-force_third+force_fourth)*xyzoverr
        
        # SageMath expression
        # force2 = B*ac*np.exp(-r*ac) + A*av*np.exp(-r*av) + 6*C/(r**7*(np.exp(-20*r/R + 20) + 1)) - 20*C*np.exp(-20*r/R + 20)/(r**6*R*(np.exp(-20*r/R + 20) + 1)**2)
        # force2 *= xyzoverr

        #  print('Force {}'.format(i+2))
        #  print force
        
        total_force += force

    return {'pot': total_pot,
            'force': np.round(total_force, 12)}

In [ ]:
def manual(parameters, atoms, molname='ft1', slabl=(1, 2)):
    # IN: parameters and atom
    #     which molecule it is and its bounds
    # OUT: Energy, Forces
    def all_distances(atoms):
        # takes one geometry as an input
        # returns the distance of each molecular atom to all slab atoms
        # return ndarray(60, 1568, 3)
        
        # ft1: slab = atoms[60:60+1344]
        
        slab = atoms[slabl[0]:slabl[0]+slabl[1]]
        mol = atoms[:slabl[0]]+atoms[slabl[0]+slabl[1]:]
        
        slab_pos = slab.get_positions()
        mol_pos = mol.get_positions()
        
        all_distances = list()
        for m, mol_atom in enumerate(mol_pos):
            distances = list()
            for slab_atom in slab_pos:
                distances.append(slab_atom-mol_atom)

            all_distances.append(np.array(distances))

        return np.array(all_distances)*bohr

    slab = atoms[slabl[0]:slabl[0]+slabl[1]]
    mol = atoms[:slabl[0]]+atoms[slabl[0]+slabl[1]:]

    slab_pos = slab.get_positions()
    mol_pos = mol.get_positions()

    all_potentials = np.empty(len(mol))
    all_forces = np.empty((len(mol), 3))
    
    if molname == 'ft1':
        pf = [0]*24+[1]*18+[2]*(18+224)
    elif molname == 'ft2':
        pf = [1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1,
              0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1,]+[2]*(18+224)
    for i, r in enumerate(all_distances(atoms)):
        # ft1:
        # 24 C_C
        # 18 C_H
        # 18 H
        # [0]*24+[1]*18+[2]*18
        comp = potential(r, *parameters[6*pf[i]:6*pf[i]+6])
        all_potentials[i] = comp['pot']
        all_forces[i] = comp['force']

    all_potentials = np.sum(np.array(all_potentials))
    all_forces = np.array(all_forces)

    return {
        'pot': all_potentials,
        'force': all_forces
    }

## Define objective function

In [ ]:
def rms(parameters,
        forces_fac1, energy_fac1, atoms1,
        forces_fac2, energy_fac2, atoms2):
    global iterations
    global difdiff

#    with open(folder+'/output.txt', 'a') as f:
#        f.write('Call #{} to rms\n'.format(iterations))
#        f.write(str(parameters))
#        f.write('\n')
    time_superstart = time.time()
    print('Call #{} to rms\n'.format(iterations))
    print(str(parameters))
    print('\n')

    # There are 60 ft1 and 60 ft2
    mm1_forces = []
    mm1_pot = []
    for i, a in enumerate(atoms1):
        time_start = time.time()
        print "{} - {}".format(i+1, str(int(time_start))[-3:])
        man = manual(parameters, a, molname='ft1', slabl=(60, 1344+224))
        mm1_forces.append(man['force'])
        mm1_pot.append(man['pot'])
        time_end = time.time()
        print "took {}s".format(np.round(time_end-time_start, 2))
        
    mm1_forces = np.asarray(mm1_forces)
    mm1_pot = np.asarray(mm1_pot)

    force_dif1 = mm1_forces + forces_fac1
    pot_dif1 = mm1_pot + energy_fac1

    # There are 60 ft1 and 60 ft2

    mm2_forces = []
    mm2_pot = []
    for i, a in enumerate(atoms2):
        time_start = time.time()
        print "{} - {}".format(i+1, str(int(time_start))[-3:])
        man = manual(parameters, a, molname='ft2', slabl=(46, 1344+224))
        mm2_forces.append(man['force'])
        mm2_pot.append(man['pot'])
        time_end = time.time()
        print "took {}s".format(np.round(time_end-time_start, 2))
        
    mm2_forces = np.asarray(mm2_forces)
    mm2_pot = np.asarray(mm2_pot)

    force_dif2 = mm2_forces - forces_fac2
    pot_dif2 = mm2_pot - energy_fac2

    time_superend = time.time()
    print 'Supertime: {}'.format(np.round(time_superend-time_superstart, 2))
    
    print 'Calculate Target'
    # Target function:
    # sum(force_dif^2)/n_atoms
    dif = np.sum(np.power(force_dif1, 2))/338.
    dif += np.sum(np.power(force_dif2, 2))/270.
    dif /= 2.
    dif += np.sum(np.power(pot_dif1, 2))/2.
    dif += np.sum(np.power(pot_dif2, 2))/2.
    difdiff.append(dif)
#    with open(folder+'/output.txt', 'a') as f:
#        f.write('Found dif = {}\n'.format(sqrtdif))
#        f.write('Change = {}'.format(difdiff[-2]-difdiff[-1]))
    print('Found dif = {}\n'.format(dif))
    print('Change = {}'.format(difdiff[-2]-difdiff[-1]))
    iterations += 1
    return dif

## Optimization

In [ ]:
difdiff = [0,0]
iterations = 0

In [ ]:
r = rms(pp,
    forces_fac1, energy_fac1, atoms1,
    forces_fac2, energy_fac2, atoms2)

print r

In [ ]:
forces_fac1.shape

# DEBUG

## SAGE THIS
```python
var('A av B ac C R r x y z DFTpot DFTFx DFTFy DFTFz')
r = sqrt(x^2+y^2+z^2)

potential(x,y,z) = A*e^(-av*r) + B*e^(-ac*r) + C/r**6/(1+exp(-20*(r/R-1)))
Fx = diff(potential, x)
Fy = diff(potential, y)
Fz = diff(potential, z)
```